# Лабораторная работа 1
## Тема: «Численные методы решения задачи Коши для ОДУ»
> Пашкевич Сергей\
> Вариант 7

$$
u''(t) + u'(t)\tan t - u(t)\cos^2t = 0\\
0 \leq t \leq 1\ \ \ u(0)=1,\ \ \ u'(0)=-1\\
u(t) = e^{-\sin t}
$$
- Явный метод Рунге-Кутта 4-го порядка; 
- Экстраполяционный метод Адамса 4-го порядка; 
- Неявный метод трапеций.

Сразу сделаем замену $\ \ u_1 = u\ \ u_2 = u'$
Тогда получаем систему:
$$
\begin{cases}
u_1'(t) = u_2(t) \\
u_2'(t) = u_1\cos^2t - u_2\tan t\\
u_1(0) = 1,\ \ u_2(0) = -1,\ \ t\in[0,1]
\end{cases}
$$
$$\omega_{\tau} = \{\tau\cdot j\ |\ j = \overline{0, 20}\}$$

Объявим переменные и функции, которые нам понадобяться в дальнейшем

In [1]:
# Импорт необходимых библиотек
import numpy as np 
import pandas as pd

a, b, tau = 0, 1, 0.05
# Начальные условия
y10, y20 = 1, -1

# Функции (не сложно догадаться откуда они)
def f1(t, u1, u2):
    return u2

def f2(t, u1, u2):
    return u1 * np.cos(t) ** 2 - u2 * np.tan(t)

# Эталонная функция
def u(t):
    return np.exp(-np.sin(t))

# И её производная:
def Du(t):
    return -np.cos(t)*np.exp(-np.sin(t))

***
### 1. Явный метод Рунге-Кутта 4-ого порядка
Формулы для подсчёта:
$$ u_1 \approx y^{(1)}\ \ u_2 \approx y^{(2)} $$

$$ y^{(1)}_{j+1} = y^{(1)}_j + \frac{\tau}6(k_1 + 2k_2 + 2k_3 + k_4)$$

$$ y^{(2)}_{j+1} = y^{(2)}_j + \frac{\tau}6(q_1 + 2q_2 + 2q_3 + q_4)$$

$$ k_1 = f_1\left(t_j, y^{(1)}_j, y^{(2)}_j\right) $$
$$ q_1 = f_2\left(t_j, y^{(1)}_j, y^{(2)}_j\right) $$

$$ k_2 = f_1\left(t_j + \frac{\tau}2, y^{(1)}_j + \frac{\tau}2k_1, y^{(2)}_j + \frac{\tau}2q_1\right) $$
$$ q_2 = f_2\left(t_j + \frac{\tau}2, y^{(1)}_j + \frac{\tau}2k_1, y^{(2)}_j + \frac{\tau}2q_1\right) $$

$$ k_3 = f_1\left(t_j + \frac{\tau}2, y^{(1)}_j + \frac{\tau}2k_2, y^{(2)}_j + \frac{\tau}2q_2\right) $$
$$ q_3 = f_2\left(t_j + \frac{\tau}2, y^{(1)}_j + \frac{\tau}2k_2, y^{(2)}_j + \frac{\tau}2q_2\right) $$

$$ k_4 = f_1\left(t_j + \tau, y^{(1)}_j + \tau k_3, y^{(2)}_j + \tau q_3\right) $$
$$ q_4 = f_2\left(t_j + \tau, y^{(1)}_j + \tau k_3, y^{(2)}_j + \tau q_3\right) $$

Данный метод имеет 4-ую степень точности

In [2]:
def runge_kutta(a, b, tau):
    
    w = np.arange(a, b + tau, tau)
    [y1, y2] = np.empty([2, w.size])
    y1[0], y2[0] = y10, y20
    
    for j in range(y1.size-1):
        
        k1 = f1(w[j], y1[j], y2[j])
        q1 = f2(w[j], y1[j], y2[j])
        
        k2 = f1(w[j] + tau/2, y1[j] + tau/2 * k1, y2[j] + tau/2 * q1)
        q2 = f2(w[j] + tau/2, y1[j] + tau/2 * k1, y2[j] + tau/2 * q1)
        
        k3 = f1(w[j] + tau/2, y1[j] + tau/2 * k2, y2[j] + tau/2 * q2)
        q3 = f2(w[j] + tau/2, y1[j] + tau/2 * k2, y2[j] + tau/2 * q2)
        
        k4 = f1(w[j] + tau, y1[j] + tau * k2, y2[j] + tau * q2)
        q4 = f2(w[j] + tau, y1[j] + tau * k2, y2[j] + tau * q2)
        
        y1[j+1] = y1[j] + tau/6*(k1 + 2*k2 + 2*k3 + k4)
        y2[j+1] = y2[j] + tau/6*(q1 + 2*q2 + 2*q3 + q4)
        
    return np.array([y1, y2])

***
### 2. Экстраполяционный метод Адамса 4-го порядка

Первые 4 значения найдём при помощи уже выше написанного метода Рунге-Кутта.
Значения функций в последующих узлах будем считать по формулам:
$$(f^{(i)} = f(t_i, y^{(1)}_i, y^{(2)}_i) $$

$$ y^{(1)}_j = y^{(1)}_{j-1} + \frac{\tau}{24}\left(55 f^{(j-1)}_1 - 59 f^{(j-2)}_1 + 37f^{(j-3)}_1 - 9f^{(j-4)}_1 \right)$$

$$ y^{(2)}_j = y^{(2)}_{j-1} + \frac{\tau}{24}\left(55 f^{(j-1)}_2 - 59 f^{(j-2)}_2 + 37f^{(j-3)}_2 - 9f^{(j-4)}_2 \right)$$

$$j = \overline{4, N} $$
Он тоже имеет 4-ую степень точности.

In [3]:
def adams_method(a, b, tau):
    
    w = np.arange(a, b + tau, tau)
    [y1, y2] = np.empty([2, w.size])
    y1[0], y2[0] = y10, y20
    [y1[:4], y2[:4]] = runge_kutta(a, a + tau*3, tau)
    ancillary = tau/24 * np.array([-9, 37, -59, 55])
    
    for j in range(4, w.size):
        y1[j] = y1[j-1] + ancillary.dot(f1(w[j-4:j], y1[j-4:j], y2[j-4:j]))
        y2[j] = y2[j-1] + ancillary.dot(f2(w[j-4:j], y1[j-4:j], y2[j-4:j]))
        
    return np.array([y1, y2])   

***
### 3. Неявный метод трапеций
Чтобы не решать систему уравнений, будем находить какое-то приближённое, промежуточное значение при помощи явного метода Эйлера.
Будем использовать ниже приведённые формулы:

$$ y^{(1)}_{j+1} = y^{(1)}_j + \frac{\tau}2\left(f_1(t_j, y^{(1)}_j, y^{(2)}_j) + f_1(t_{j+1}, \widetilde y^{(1)}_{j+1}, \widetilde y^{(2)}_{j+1})\right) $$

$$ y^{(2)}_{j+1} = y^{(2)}_j + \frac{\tau}2\left(f_2(t_j, y^{(1)}_j, y^{(2)}_j) + f_2(t_{j+1}, \widetilde y^{(1)}_{j+1}, \widetilde y^{(2)}_{j+1})\right) $$

$$ \widetilde y^{(1)}_{j+1} = y^{(1)}_j + \tau f_1(t_j, y^{(1)}_j, y^{(2)}_j) $$

$$ \widetilde y^{(2)}_{j+1} = y^{(2)}_j + \tau f_2(t_j, y^{(1)}_j, y^{(2)}_j) $$

In [4]:
def traps(a, b, tau):
    
    w = np.arange(a, b + tau, tau)
    [y1, y2] = np.empty([2, w.size])
    y1[0], y2[0] = y10, y20
    
    for j in range(w.size-1):
        
        t1 = y1[j] + tau*f1(w[j], y1[j], y2[j])
        t2 = y2[j] + tau*f2(w[j], y1[j], y2[j])
        
        y1[j+1] = y1[j] + tau/2*( f1(w[j], y1[j], y2[j]) + f1(w[j+1], t1, t2))
        y2[j+1] = y2[j] + tau/2*( f2(w[j], y1[j], y2[j]) + f2(w[j+1], t1, t2))
        
    return np.array([y1, y2])

***
## Результаты и оценки:

Для оценки погрешности будем использовать правило Рунге. Подобной оценки, по условию, требует первый и последний метод. Первый метод (метод Рунге-Кутта) имеет порядок точности, потому будем использовать следующую формулу для оценки:
$$ \max_{i\in \overline{1, N}}\frac{|y_{i,2\tau} - y_{2i,\tau}|}{2^p-1}
 = \frac1{15}\max_{i\in \overline{1, N}}|y_{i,2\tau} - y_{2i,\tau}| $$

In [9]:
runge1 = np.max(np.abs(runge_kutta(a, b, tau)[::,::2] - runge_kutta(a, b, 2*tau)))/15

Для неявного метода трапеций всё аналогично, только $p=2$:

In [10]:
runge2 = np.max(np.abs(traps(a, b, tau)[::,::2] - traps(a, b, 2*tau)))/3

Таблица 1:

In [7]:
a, b, tau = 0, 1, 0.05
w = np.arange(a, b + tau, tau)

pd.DataFrame(np.array([
    w, u(w), Du(w),
    *runge_kutta(a, b, tau),
    *adams_method(a, b, tau),
    *traps(a, b, tau)
]).T, columns=['$t_j$', '$u(t_j)$', '$u\'(t_j)$', 
               'meth1 $y_1(t_j)$', 'meth1 $y_2(t_j)$', 
               'meth2 $y_1(t_j)$', 'meth2 $y_2(t_j)$', 
               'meth3 $y_1(t_j)$', 'meth3 $y_2(t_j)$'
              ])

,$t_j$,$u(t_j)$,$u'(t_j)$,meth1 $y_1(t_j)$,meth1 $y_2(t_j)$,meth2 $y_1(t_j)$,meth2 $y_2(t_j)$,meth3 $y_1(t_j)$,meth3 $y_2(t_j)$
0,0.00,1.000000,-1.000000,1.000000,-1.000000,1.000000,-1.000000,1.000000,-1.000000
1,0.05,0.951249,-0.950060,0.951249,-0.950060,0.951249,-0.950060,0.951250,-0.950121
2,0.10,0.904988,-0.900467,0.904988,-0.900467,0.904988,-0.900467,0.904989,-0.900584
3,0.15,0.861192,-0.851521,0.861191,-0.851521,0.861191,-0.851521,0.861193,-0.851690
4,0.20,0.819821,-0.803479,0.819820,-0.803479,0.819819,-0.803478,0.819822,-0.803695
5,0.25,0.780825,-0.756551,0.780824,-0.756551,0.780823,-0.756550,0.780825,-0.756811
6,0.30,0.744144,-0.710908,0.744143,-0.710908,0.744141,-0.710906,0.744143,-0.711207
7,0.35,0.709711,-0.666683,0.709709,-0.666682,0.709707,-0.666679,0.709706,-0.667016
8,0.40,0.677451,-0.623974,0.677449,-0.623973,0.677447,-0.623969,0.677443,-0.624337
9,0.45,0.647287,-0.582848,0.647285,-0.582847,0.647283,-0.582842,0.647274,-0.583238


Таблица два:

In [11]:
pd.DataFrame([
        [runge1, '-', runge2],
        [np.max(np.abs(f(a, b, tau) - [u(w), Du(w)])) for f in (runge_kutta, adams_method, traps)]
    ],
    columns=['Метод 1', 'Метод 2', 'Метод 3'],
    index=['Оценка погрешности по правилу Рунге', 
             '$$\max_{j=\overline{1,N}}\left(|u(t_j)-y_1(t_j)|, |u(t_j)-y_2(t_j)|\right)$$']
)

,Метод 1,Метод 2,Метод 3
Оценка погрешности по правилу Рунге,0.000002,-,0.000482
"$$\max_{j=\overline{1,N}}\left(|u(t_j)-y_1(t_j)|, |u(t_j)-y_2(t_j)|\right)$$",0.000004,9.43723e-06,0.000472
